# Exploring a DB with natural language
--------

## 1. What we learned in last class, Let us recall
- Understanding data of hotels db
- creating interpreter

In [1]:
import os
os.chdir("C:\\Users\\ramreddymyla\\Google Drive\\01 DS ML DL NLP and AI With Python Lab Copy\\02 Lab Data\\Python")

In [2]:
# Step 1 of 7: Import required packages
from sqlalchemy import create_engine ; import pandas as pd ;
# Step 2 of 7: Create engine: engine
engine = create_engine('sqlite:///data/hotels.db') 
# Step 3 of 7: Open engine connection
con = engine.connect()
# Step 4 of 7: Perform query: result
result = con.execute("SELECT * FROM hotels") 
# Step 5 of 7: Save results of the query to DataFrame: df
df = pd.DataFrame(result.fetchall(),columns = result.keys())
# Step 6 of 7: Close connection
con.close() 
# Step 7 of 7: Print head of DataFrame df 
print(df)

               name price    area  stars
0    Hotel for Dogs   mid    east      3
1  Hotel California   mid   north      3
2       Grand Hotel    hi   south      5
3      Cozy Cottage    lo   south      2
4         Ben's BnB    hi   north      4
5         The Grand    hi    west      5
6     Central Rooms   mid  center      3


In [3]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
# Create args dictionary
args = {"pipeline":"tensorflow_embedding"}
# Create a configuration and trainer
config = RasaNLUModelConfig(configuration_values=args)
trainer = Trainer(config)
# Load the training data
training_data = load_data("data/Hotels_DB_NLU.json")
# Create an interpreter by training the model
interpreter = trainer.train(training_data)

C:\Users\ramreddymyla\Anaconda3\envs\rasax\lib\site-packages\rasa_nlu\__init__.py:12: UserWarning: The 'rasa_nlu' package has been renamed. You should change your imports to use 'rasa.nlu' instead.
  UserWarning,
You have specified the pipeline template 'tensorflow_embedding' which has been renamed to 'supervised_embeddings'. Please update your code as it will no longer work with future versions of Rasa NLU.
C:\Users\ramreddymyla\Anaconda3\envs\rasax\lib\site-packages\rasa\nlu\training_data\training_data.py:209: UserWarning: Entity '' has only 1 training examples! minimum is 2, training may fail.
  "".format(entity_type, count, self.MIN_EXAMPLES_PER_ENTITY)


Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Epochs: 100%|██████████| 300/300 [00:25<00:00, 11.64it/s, loss=0.007, acc=1.000]


In [4]:
# Try it out
import json
print(json.dumps(interpreter.parse("I want an 5 star hotel in the west of town"), indent=2))

{
  "intent": {
    "name": "restaurant_search",
    "confidence": 0.9664635062217712
  },
  "entities": [
    {
      "start": 10,
      "end": 11,
      "value": "5",
      "entity": "stars",
      "confidence": 0.994618347140683,
      "extractor": "CRFEntityExtractor"
    },
    {
      "start": 30,
      "end": 34,
      "value": "west",
      "entity": "area",
      "confidence": 0.9111861115971476,
      "extractor": "CRFEntityExtractor"
    }
  ],
  "intent_ranking": [
    {
      "name": "restaurant_search",
      "confidence": 0.9664635062217712
    },
    {
      "name": "affirm",
      "confidence": 0.06291888654232025
    },
    {
      "name": "greet",
      "confidence": 0.0
    },
    {
      "name": "goodbye",
      "confidence": 0.0
    }
  ],
  "text": "I want an 5 star hotel in the west of town"
}


## 2.	Exercise 1: Creating queries from parameters
- Now you're going to implement a more powerful function for querying the hotels database. The goal is to take arguments that can later be specified by other parts of your code.
- Specifically, your job here is to define a find_hotels() function which takes a single argument - a dictionary of column names and values - and returns a list of matching hotels from the database.

In [15]:
import sqlite3

In [17]:
# Define find_hotels()
def find_hotels(params): # pass param argument in dict format # params = {"area": "south", "price":"lo"}
    # Create the base query
    query = 'SELECT * FROM hotels'
    # Add filter clauses for each of the parameters
    if len(params) > 0: # 2>0
        filters = ["{}=?".format(k) for k in params]
        print("filters:" ,filters) # ['area=?', 'price=?']
        query += " WHERE " + " and ".join(filters) 
        print("query: ",query) # final query is created
        
    # Create the tuple of values
    t = tuple(params.values())  # ['south', 'lo']
    print(t)
        # Open connection to DB
    conn = sqlite3.connect('data/hotels.db')
    # Create a cursor
    c = conn.cursor()
    # Execute the query
    c.execute(query, t)
    # Return the results
    return c.fetchall()

## 3. Exercise 2: Using your custom function to find hotels
- Here, you're going to put your find_hotels() function into action! Recall that it accepts a single argument, **params**, which is a dictionary of column names and values.


In [18]:
# Create the dictionary of column names and values
params = {"area": "south", "price":"lo"}
# Find the hotels that match the parameters
print(find_hotels(params))


filters: ['area=?', 'price=?']
query:  SELECT * FROM hotels WHERE area=? and price=?
('south', 'lo')
[('Cozy Cottage', 'lo', 'south', 2)]


In [8]:
len(params)

2

In [9]:
params.values()

dict_values(['south', 'lo'])

## 4. Exercise 3: Creating SQL from natural language
- Now you'll write a respond() function which can handle messages like "I want an expensive hotel in the south of town" and respond appropriately according to the number of matching results in a database. This is important functionality for any database-backed chatbot.
- Your find_hotels() function from the previous exercises has already been defined for you, along with a rasa NLU interpreter object which can handle hotel queries and a list of responses, which you can explore in the Shell.

In [12]:
# response templates
responses = ["I'm sorry 😂 I couldn't find anything like that",
             '{} is a great hotel!',
             '{} or {} would work!',
             '{} is one option, but I know others too :)']

In [11]:
type(responses)

list

In [12]:
responses[0]

"I'm sorry 😂 I couldn't find anything like that"

In [22]:
# Define respond()
def respond(message): # I want an 5 star hotel in the west of town
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    print("******",entities,"***********")
    # Initialize an empty params dictionary
    params = {}
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])
    print(" Columnnames and values extracted from NLU :{}".format(params))

    # Find hotels that match the dictionary
    results = find_hotels(params)
    print("results from find_hotels function is :{} ".format(results))
    # Get the names of the hotels and index of the response
    names = [r[0] for r in results]
    print(names)    
    n = min(len(results),3)
    print(n)
    # Select the nth element of the responses array
    return responses[n].format(*names)

## Try it out :)

In [23]:
# Query Database Data based on area    
print(respond("I want an 5 star hotel in the west of town"))

****** [{'start': 10, 'end': 11, 'value': '5', 'entity': 'stars', 'confidence': 0.994618347140683, 'extractor': 'CRFEntityExtractor'}, {'start': 30, 'end': 34, 'value': 'west', 'entity': 'area', 'confidence': 0.9111861115971476, 'extractor': 'CRFEntityExtractor'}] ***********
 Columnnames and values extracted from NLU :{'stars': '5', 'area': 'west'}
filters: ['stars=?', 'area=?']
query:  SELECT * FROM hotels WHERE stars=? and area=?
('5', 'west')
results from find_hotels function is :[('The Grand', 'hi', 'west', 5)] 
['The Grand']
1
The Grand is a great hotel!


In [14]:
# Observe database data
df

,name,price,area,stars
0,Hotel for Dogs,mid,east,3
1,Hotel California,mid,north,3
2,Grand Hotel,hi,south,5
3,Cozy Cottage,lo,south,2
4,Ben's BnB,hi,north,4
5,The Grand,hi,west,5
6,Central Rooms,mid,center,3


In [16]:
# Query Database Data based on area    
print(respond("I want an 5 star hotel in the north of town"))

 Columnnames and values extracted from NLU :{'stars': '5', 'area': 'north'}
filters: ['stars=?', 'area=?']
query:  SELECT * FROM hotels WHERE stars=? and area=?
('5', 'north')
results from find_hotels function is :[] 
[]
0
I'm sorry 😂 I couldn't find anything like that


In [17]:
# Query Database Data based on area    
print(respond("I want a hotel in the north of town"))

 Columnnames and values extracted from NLU :{'area': 'north'}
filters: ['area=?']
query:  SELECT * FROM hotels WHERE area=?
('north',)
results from find_hotels function is :[('Hotel California', 'mid', 'north', 3), ("Ben's BnB", 'hi', 'north', 4)] 
['Hotel California', "Ben's BnB"]
2
Hotel California or Ben's BnB would work!


In [18]:
# Query Database Data based on area    
print(respond("I want a 3 star hotel"))

 Columnnames and values extracted from NLU :{'stars': '3'}
filters: ['stars=?']
query:  SELECT * FROM hotels WHERE stars=?
('3',)
results from find_hotels function is :[('Hotel for Dogs', 'mid', 'east', 3), ('Hotel California', 'mid', 'north', 3), ('Central Rooms', 'mid', 'center', 3)] 
['Hotel for Dogs', 'Hotel California', 'Central Rooms']
3
Hotel for Dogs is one option, but I know others too :)


# Incremental Slot Filling
------
- Incremental Filters


## Exercise 1: Refining your search

- Now you'll write a bot that allows users to add filters incrementally, in case they don't specify all of their preferences in one message.
- To do this, initialize an empty dictionary params outside of your respond() function (unlike inside the function, like in the previous exercise). Your respond() function will take in this dictionary as an argument.

In [21]:
# Define a respond function, taking the message and existing params as input
def respond(message, params):
    # Extract the entities
    entities = interpreter.parse(message)["entities"]
    # Fill the dictionary with entities
    for ent in entities:
        params[ent["entity"]] = str(ent["value"])
    # Find the hotels
    results = find_hotels(params)
    names = [r[0] for r in results]
    n = min(len(results), 3)
    # Return the appropriate response
    return responses[n].format(*names), params

In [22]:
# Initialize params dictionary
params = {}
# Pass the messages to the bot
for message in ["in the north of town","I want an 3 star hotel"]:
    print("USER: {}".format(message))
    response, params = respond(message, params)
    print("BOT: {}".format(response))

USER: in the north of town
filters: ['area=?']
query:  SELECT * FROM hotels WHERE area=?
('north',)
BOT: Hotel California or Ben's BnB would work!
USER: I want an 3 star hotel
filters: ['area=?', 'stars=?']
query:  SELECT * FROM hotels WHERE area=? and stars=?
('north', '3')
BOT: Hotel California is a great hotel!


In [23]:
df

,name,price,area,stars
0,Hotel for Dogs,mid,east,3
1,Hotel California,mid,north,3
2,Grand Hotel,hi,south,5
3,Cozy Cottage,lo,south,2
4,Ben's BnB,hi,north,4
5,The Grand,hi,west,5
6,Central Rooms,mid,center,3
